**Install Gymnasium**

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.



**Import**



In [2]:
import os; import random; import numpy as np; from collections import namedtuple, deque
import torch; from torch import nn; from torch import optim; from torch.nn import functional as F; from torch import autograd; from torch.autograd import Variable;

Create the architecture of neural network

In [3]:
class Network(nn.Module):
  def __init__(self, state_size, action_size, seed=42):
    super().__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, 64)
    self.fc2 = nn.Linear(64, 64)
    self.fc3 = nn.Linear(64, action_size)

  def forward(self, state):
    x = self.fc1(state); x = F.relu(x)
    x = self.fc2(x); x = F.relu(x)
    return self.fc3(x)


**Setting up train environment**

In [4]:
import gymnasium as gym
env = gym.make('LunarLander-v2')
state_shape = env.observation_space.shape; state_size = state_shape[0]; number_actions = env.action_space.n
print(f'state_shape: {state_shape}\nstate_size: {state_size}\nnumber_actions: {number_actions}')

state_shape: (8,)
state_size: 8
number_actions: 4


**Initializing the hyperparameters**

In [5]:
learning_rate = 5e-4; minibatch_size=100; discount_factor = 0.99; replay_buffer_size = int(1e5); interpolation_parameter = 1e-3

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
class ReplayMemory:
  def __init__(self, capacity):
    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    self.capacity = capacity
    self.memory = list()

  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  def sample(self, batch_size):
    experiences = random.sample(self.memory, k = batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None])).to(torch.int8).float().to(self.device)
    return states, next_states, actions, rewards, dones

**Implements DQN**

In [7]:
class Agent:
  def __init__(self, state_size, action_size):
    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    self.state_size = state_size; self.action_size = action_size
    self.local_qnetwork = Network(self.state_size, self.action_size).to(self.device)
    self.target_qnetwork = Network(self.state_size, self.action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = ReplayMemory(replay_buffer_size)
    self.t_step = 0

  def step(self, state, action, reward, next_action, done):
    self.memory.push((state, action, reward, next_action, done))
    self.t_step = (self.t_step + 1) % 4
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)

  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().numpy())
    return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + (discount_factor * next_q_targets * (1-dones))
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad(); loss.backward(); self.optimizer.step(); self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param)



**Train DQN**

In [8]:
agent = Agent(state_size, number_actions)

In [9]:
number_episodes = 2000; maximum_number_timesteps_per_episode = 1000
epsilon_starting_value = 1.0; epsilon_ending_value = 0.01; epsilon_decay_value = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)
for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state; score += reward
    if done:
      break
  scores_on_100_episodes.append(score); epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisoles: {}\tAverage score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end="")
  if episode % 100 == 0:
    print('\rEpisoles: {}\tAverage score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Episoles: 100	Average score: -162.53
Episoles: 200	Average score: -118.66
Episoles: 300	Average score: -63.89
Episoles: 400	Average score: -35.91
Episoles: 500	Average score: -32.83
Episoles: 600	Average score: 42.37
Episoles: 700	Average score: 177.60
Episoles: 800	Average score: 175.92
Episoles: 900	Average score: 177.05
Episoles: 990	Average score: 201.98
Environment solved in 890 episodes!	Average score: 201.98


In [13]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder


def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False; frames = list()
    while not done:
        frame = env.render(); frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alg="test" autoplay loop controls style="height: 400px;"><source src="data:video/mp4;base64,{0}", type="video/mp4" /></video>'''.format(encoded.decode('ascii'))))
    else:
        print('Could not find video')

In [14]:
show_video()